In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os

path = "./data/상해보험"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

file_list: ['24599_3_1[0].pdf', '24600_3_1.pdf', '24602_3_1.pdf', '24603_3_1.pdf', '24605_3_1.pdf', '24606_3_1.pdf', '24608_3_1.pdf', '24609_3_1.pdf', '24610_3_1.pdf', '24611_3_1(일반).pdf']


In [62]:
# model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # pool_mask 생성 (입력 시퀀스 길이에 맞춰서 1로 설정)
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)  # pool_mask 추가

    return outputs.embedding.numpy()

In [ ]:
import faiss
import pickle
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

# 1️⃣ FAISS 저장 경로
faiss_index_path = "./faiss_index_1.bin"
metadata_path = "./documents_1.pkl"

# 2️⃣ 임베딩 모델 로드 (Sentence-BERT 사용)
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


# ## 변환
# def get_embeddings(texts):
#     """텍스트 리스트를 입력받아 임베딩을 반환하는 함수"""
#     return embedding_model.encode(texts, convert_to_numpy=True)

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.numpy()


# 3️⃣ PDF 문서 로드
documents = []  # 모든 문서를 저장할 리스트
for file in file_list:  # 첫 번째 파일만 로드
    loader = PyPDFLoader(path + "\\" + file)
    documents.extend(loader.load())  # ✅ 여러 문서를 리스트에 추가

# 4️⃣ 문서 전처리
for doc in documents:
    doc.page_content = doc.page_content.replace("\n", " ").replace("  ", " ")
    doc.metadata["source"] = file  # ✅ 문서의 출처 정보를 추가

# 5️⃣ 문서 스플리팅
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
split_documents = splitter.split_documents(documents)  # ✅ 리스트 전체를 입력해야 함

# 6️⃣ 스플릿된 문서로 벡터 생성
batch_size = 16
texts = [doc.page_content for doc in split_documents]
embeddings = []
# print(texts)

for i in range(0, len(texts), batch_size):
    batch = texts[i : i + batch_size]
    batch_embeddings = get_embeddings(batch)  # 🔥 배치 단위로 임베딩 생성
    embeddings.extend(batch_embeddings)

# 7️⃣ FAISS 인덱스 생성 및 저장
embedding_dim = len(embeddings[0])
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings, dtype=np.float32))

faiss.write_index(index, faiss_index_path)
with open(metadata_path, "wb") as f:
    pickle.dump(split_documents, f)

print("✅ FAISS 인덱스 및 문서 저장 완료!")

In [ ]:
# 코드 저장됬는지 확인
import faiss
import pickle

# 저장된 인덱스 & 문서 불러오기
index = faiss.read_index("./faiss_index_1.bin")

with open("./documents_1.pkl", "rb") as f:
    split_documents = pickle.load(f)

print("✅ FAISS 인덱스 로드 완료!")
print(f"🔹 저장된 문서 개수: {len(split_documents)}")

✅ FAISS 인덱스 로드 완료!
🔹 저장된 문서 개수: 6488
